# 강화학습 - 카트폴
## Matthias Plappert 라이브러리 사용
## PC에서 실행


In [0]:
!pip install keras

In [0]:
import keras

Using TensorFlow backend.
/Users/hwa-kim/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/hwa-kim/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/hwa-kim/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/hwa-kim/opt/anaconda3/lib/pytho

In [1]:
!git clone https://github.com/matthiasplappert/keras-rl.git 

Cloning into 'keras-rl'...
remote: Enumerating objects: 1723, done.
remote: Total 1723 (delta 0), reused 0 (delta 0), pack-reused 1723
Receiving objects: 100% (1723/1723), 1.38 MiB | 1.11 MiB/s, done.
Resolving deltas: 100% (1070/1070), done.


In [2]:
%cd keras-rl

/content/keras-rl


In [3]:
!pwd

/content/keras-rl


In [4]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating keras_rl.egg-info
writing keras_rl.egg-info/PKG-INFO
writing dependency_links to keras_rl.egg-info/dependency_links.txt
writing requirements to keras_rl.egg-info/requires.txt
writing top-level names to keras_rl.egg-info/top_level.txt
writing manifest file 'keras_rl.egg-info/SOURCES.txt'
writing manifest file 'keras_rl.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/tests
copying tests/__init__.py -> build/lib/tests
creating build/lib/rl
copying rl/callbacks.py -> build/lib/rl
copying rl/policy.py -> build/lib/rl
copying rl/__init__.py -> build/lib/rl
copying rl/random.py -> build/lib/rl
copying rl/memory.py -> build/lib/rl
copying rl/processors.py -> build/lib/rl
copying rl/core.py -> build/lib/rl
copying rl/util.py -> build/lib/rl
creating build/lib/utils
copying utils/__init__.py -> build/lib/utils
creating

In [5]:
!pip install gym
!pip install gym[atarti]
!pip install gym[Box2D]

     |████████████████████████████████| 450kB 11.5MB/s 


In [6]:
import numpy as np
import gym

from keras.models import Model
from keras.layers import Dense, Flatten, Input
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.callbacks import ModelIntervalCheckpoint, FileLogger


Using TensorFlow backend.


ModuleNotFoundError: ignored

In [0]:
def build_model(state_size, num_actions):
    input = Input(shape=(1,state_size))
    x = Flatten()(input)
    x = Dense(16, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    output = Dense(num_actions, activation='linear')(x)
    model = Model(inputs=input, outputs=output)
    print(model.summary())
    return model


def build_callbacks(env_name):
    checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
    log_filename = 'dqn_{}_log.json'.format(env_name)
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, 
                                         interval=5000)]
    callbacks += [FileLogger(log_filename, interval=100)]
    return callbacks

In [0]:
# 카트폴 환경설정
ENV_NAME = 'CartPole-v0'
env = gym.make(ENV_NAME)
np.random.seed(9)
env.seed(9)
num_actions = env.action_space.n
state_space = env.observation_space.shape[0]
print(num_actions)

model = build_model(state_space, num_actions)

# 에이전트의 경험을 저장한 빠른 메모리구조, 최대크기 지정
memory = SequentialMemory(limit=50000, window_length=1)

# e 탐욕 Q 정책 설정(1~0.1, 10000 스텝)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), 
           attr='eps', value_max=1., value_min=.1, value_test=.05,
           nb_steps=10000)

# 에이전트 생성 ()
dqn = DQNAgent(model=model, nb_actions=num_actions, 
               memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
# 컴파일
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
callbacks = build_callbacks(ENV_NAME)

dqn.fit(env, nb_steps=50000,
        visualize=False,
        verbose=0,
        callbacks=callbacks)

# After training is done, we save the final weights.
dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

2
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 4)              0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                80        
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 34        
Total params: 658
Trainable params: 658
Non-trainable params: 0
___________________________________________________________

/Users/hwa-kim/Dropbox/lab/book5/keras-rl/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


In [0]:
dqn.test(env, nb_episodes=20, visualize=True)

Testing for 20 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200


In [0]:
!ls

CONTRIBUTING.md                      dqn_CartPole-v0_weights_30000.h5f
ISSUE_TEMPLATE.md                    dqn_CartPole-v0_weights_35000.h5f
LICENSE                              dqn_CartPole-v0_weights_40000.h5f
README.md                            dqn_CartPole-v0_weights_45000.h5f
assets                               dqn_CartPole-v0_weights_5000.h5f
build                                dqn_CartPole-v0_weights_50000.h5f
dist                                 examples
docs                                 keras_rl.egg-info
dqn_CartPloe_model_architecture.json mkdocs.yml
dqn_CartPole-v0_log.json             pytest.ini
dqn_CartPole-v0_weights.h5f          rl
dqn_CartPole-v0_weights_10000.h5f    setup.cfg
dqn_CartPole-v0_weights_15000.h5f    setup.py
dqn_CartPole-v0_weights_20000.h5f    tests
dqn_CartPole-v0_weights_25000.h5f    utils


In [0]:
# with open('dqn_CartPloe_model_architecture.json', 'w') as f:
#     f.write(dqn.to_json())

In [0]:
dqn2 = DQNAgent(model=model, nb_actions=num_actions, 
               memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
# 컴파일
dqn2.compile(Adam(lr=1e-3), metrics=['mae'])

In [0]:
dqn2.load_weights('dqn_CartPole-v0_weights_50000.h5f')

In [0]:
dqn2.test(env, nb_episodes=20, visualize=True)

Testing for 20 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
